In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OrdinalEncoder

# Load datasets
train = pd.read_csv('../api/lib/data/train.csv')
test = pd.read_csv('../api/lib/data/test.csv')


In [36]:
# Define target variable
target = train['SalePrice']

# Drop target variable from train dataset
train = train.drop(columns=['SalePrice'])

# Concatenate train and test datasets
data = pd.concat([train, test])

# Get list of numerical columns
num_cols = data.select_dtypes(include=['int64', 'float64']).columns

# Get list of categorical columns
cat_cols = data.select_dtypes(include=['object']).columns

# Impute missing values here
from sklearn.impute import SimpleImputer

# For numerical columns
num_imputer = SimpleImputer(strategy='mean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

# For categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

# Perform one-hot encoding or ordinal encoding here
# Create an instance of OrdinalEncoder
encoder = OrdinalEncoder()

# Fit and transform the data
data = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

# Split the data back into train and test datasets
train = data[:len(train)]
test = data[len(train):]

# Split the train dataset into training and validation datasets
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
test = scaler.transform(test)


In [37]:
# Define model
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, batch_size=32)


Epoch 1/1000
37/37 [==============================] - 1s 8ms/step - loss: 38884765696.0000 - val_loss: 39651860480.0000
Epoch 2/1000
37/37 [==============================] - 0s 4ms/step - loss: 38880776192.0000 - val_loss: 39644631040.0000
Epoch 3/1000
37/37 [==============================] - 0s 3ms/step - loss: 38867963904.0000 - val_loss: 39622807552.0000
Epoch 4/1000
37/37 [==============================] - 0s 3ms/step - loss: 38834548736.0000 - val_loss: 39572819968.0000
Epoch 5/1000
37/37 [==============================] - 0s 3ms/step - loss: 38765850624.0000 - val_loss: 39475634176.0000
Epoch 6/1000
37/37 [==============================] - 0s 3ms/step - loss: 38641717248.0000 - val_loss: 39313850368.0000
Epoch 7/1000
37/37 [==============================] - 0s 3ms/step - loss: 38444535808.0000 - val_loss: 39067013120.0000
Epoch 8/1000
37/37 [==============================] - 0s 3ms/step - loss: 38159237120.0000 - val_loss: 38715154432.0000
Epoch 9/1000
37/37 [====================

In [39]:
from math import sqrt

# Make predictions on validation dataset
y_val_pred = model.predict(X_val)

# Calculate Mean Squared Error on validation dataset
mse = mean_squared_error(y_val, y_val_pred)
print(f'Validation MSE: {mse}')
print(f'Validation RMSE: {sqrt(mse)}')

# Make predictions on test dataset
y_test_pred = model.predict(test)


10/10 [==============================] - 0s 1ms/step
Validation MSE: 1160701148.4790943
Validation RMSE: 34069.06439101453
46/46 [==============================] - 0s 1ms/step
